# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../Resources/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,11.01,95,100,2.68,NZ,1724598050
1,1,sydney,-33.8679,151.2073,19.83,66,0,8.23,AU,1724598215
2,2,hilo,19.7297,-155.0900,25.06,100,100,8.23,US,1724598043
3,3,yellowknife,62.4560,-114.3525,11.29,82,20,1.54,CA,1724598173
4,4,wailua homesteads,22.0669,-159.3780,24.44,89,75,8.23,US,1724598137


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True,
    tiles = "OSM",
    size='Humidity',  # Size of the points based on humidity
    color='City',     # Optionally color by city for better distinction
    hover_cols=['city', 'humidity'],  # Information to display on hover
    title='City Map with Humidity as Point Size'
)

map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
ideal_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
ideal_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
ideal_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
ideal_df = city_data_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,11.01,95,100,2.68,NZ,1724598050
1,1,sydney,-33.8679,151.2073,19.83,66,0,8.23,AU,1724598215
2,2,hilo,19.7297,-155.0900,25.06,100,100,8.23,US,1724598043
3,3,yellowknife,62.4560,-114.3525,11.29,82,20,1.54,CA,1724598173
4,4,wailua homesteads,22.0669,-159.3780,24.44,89,75,8.23,US,1724598137


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,waitangi,NZ,-43.9535,-176.5597,95,
1,sydney,AU,-33.8679,151.2073,66,
2,hilo,US,19.7297,-155.0900,100,
3,yellowknife,CA,62.4560,-114.3525,82,
4,wailua homesteads,US,22.0669,-159.3780,89,
...,...,...,...,...,...,...
575,cronulla,AU,-34.0500,151.1500,66,
576,taoudenni,ML,22.6783,-3.9836,17,
577,saldanha,ZA,-33.0117,17.9442,68,
578,okha,RU,53.5739,142.9478,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
        'categories': 'accommodation.hotel',
        'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
waitangi - nearest hotel: Hotel Chathams
sydney - nearest hotel: The Fullerton Hotel
hilo - nearest hotel: Dolphin Bay Hotel
yellowknife - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
port-aux-francais - nearest hotel: Keravel
tucurui - nearest hotel: No hotel found
vilyuchinsk - nearest hotel: Победа
ribeira grande - nearest hotel: Hospedaria JSF
chibougamau - nearest hotel: No hotel found
kuanda - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
khandyga - nearest hotel: Апельсин
al bawiti - nearest hotel: Old Oasis Hotel
cidreira - nearest hotel: Hotel Castelo
port mathurin - nearest hotel: Escale Vacances
port lincoln - nearest hotel: Boston Hotel
margate - nearest hotel: The Reading Rooms
vyaz'ma - nearest hotel: Золотой улей
puente alto - nearest hotel: Hotel Mary Louis
la passe - nearest hotel: L'Estuaire
namibe - nearest hotel: Nelsal Pensao
badger - nearest hotel: Clarion Hotel & S

,City,Country,Lat,Lng,Humidity,Hotel Name
0,waitangi,NZ,-43.9535,-176.5597,95,Hotel Chathams
1,sydney,AU,-33.8679,151.2073,66,The Fullerton Hotel
2,hilo,US,19.7297,-155.0900,100,Dolphin Bay Hotel
3,yellowknife,CA,62.4560,-114.3525,82,No hotel found
4,wailua homesteads,US,22.0669,-159.3780,89,Hilton Garden Inn Kauai Wailua Bay
...,...,...,...,...,...,...
575,cronulla,AU,-34.0500,151.1500,66,No hotel found
576,taoudenni,ML,22.6783,-3.9836,17,No hotel found
577,saldanha,ZA,-33.0117,17.9442,68,Hoedjesbaai
578,okha,RU,53.5739,142.9478,76,Сфера


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True,
    tiles = "OSM",
    size='Humidity',  # Size of the points based on humidity
    color='City',     # Optionally color by city for better distinction
    hover_cols=['Hotel Name', 'Country'],  # Information to display on hover
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)